## Part I 1.3 Data Indexing
To allow for faster queries additional indices were added to some tables in the database. This includes:
* adding one to the year column in the table elite_years to speed up query_6
  * create index idx_year on elite_years(year)
* adding one to the yelping_since column in the user table
  * create index idx_yelping_since on user(yelping_since)
* creating new columns in the hours table to split the hours column into date, opening_time and closing_time and creating an index for each of these
  * create index idx_day_of_week on hours(day_of_week)
* creating new columns in the checkin table to split the date column into date_of_week and time and creating an index for each of these
  * create index idx_day_of_week on hours(day_of_week)
* creating new columns in the review table to split the date column into year, month, day columns and scrapping the time because its always 00:00:00
  * select count(*) from review where substring_index(date, ' ', -1) != '00:00:00'; #returns 0

In [1]:
"""Global functions and variables"""

import pymysql

def open_conn():
    """open the connection before each test case"""
    conn = pymysql.connect(user='public', password='ece656yelp',
                                   host='maindb.czbva1am4d4u.us-east-2.rds.amazonaws.com',
                                   database='yelp_db')
    return conn

def close_conn(conn):
    """close the connection after each test case"""
    conn.close()

def executeQuery(conn, query, commit=False):
    """ fetch result after query"""
    cursor = conn.cursor()
    query_num = query.count(";")
    if query_num > 1:
        for result in cursor.execute(query, params=None, multi=True):
            if result.with_rows:
                result = result.fetchall()
    else:
        cursor.execute(query)
        result = cursor.fetchall()
    # we commit the results only if we want the updates to the database
    # to persist.
    if commit:
        conn.commit()
    else:
        conn.rollback()
    # close the cursor used to execute the query
    cursor.close()
    return result

yelp_conn = open_conn()

In [ ]:
query_1 = "create index idx_year on elite_years(year);\
create index idx_yelping_since on user(yelping_since);\
"
executeQuery(yelp_conn, query_1)

In [ ]:
query_2 = "create table hours_new(\
    id int(11) not null auto_increment, primary key (id)\
    ,business_id varchar(255)\
    ,day_of_week varchar(9)\
    ,opening_time time\
    ,closing_time time\
    );\
    drop procedure dowhile;\
    delimiter //\
    create procedure dowhile ()\
    begin declare v1 int default 821044;\
    while v1 > 0 do\
    insert into hours_new(id) values (null);\
    set v1 = v1 - 1;\
    end while;\
    end//\
    delimiter ;\
    call dowhile();\
    update hours_new set day_of_week = (select SUBSTRING_INDEX(hours, '|', 1) from \
    hours where hours_new.id = hours.id);\
    update hours_new set opening_time = (select SUBSTRING_INDEX(SUBSTRING_INDEX(hours, '|', - 1), '-', 1) from \
    hours where hours_new.id = hours.id);\
    update hours_new set closing_time = (select SUBSTRING_INDEX(SUBSTRING_INDEX(hours, '|', - 1), '-', - 1) from \
    hours where hours_new.id = hours.id);\
    update hours_new set business_id = (select business_id from hours where hours_new.id = hours.id);"
executeQuery(yelp_conn, query_2)
query_3 =  "create index idx_day_of_week on hours(day_of_week)\
            create index idx_opening_time on hours(opening_time)\
            create index idx_closing_time on hours(closing_time)"
executeQuery(yelp_conn, query_3)

In [ ]:
query_4 = "create table checkin_new(\
    id int(11) not null auto_increment, primary key (id)\
    ,business_id varchar(255)\
    ,day_of_week varchar(9)\
    ,checkin_time time\
    ,count int(11)\
    );\
    drop procedure dowhile;\
    delimiter //\
    create procedure dowhile ()\
    begin declare v1 int default 3891600;\
    while v1 > 0 do\
    insert into checkin_new(id) values (null);\
    set v1 = v1 - 1;\
    end while;\
    end//\
    delimiter ;\
    call dowhile();\
    update checkin_new set day_of_week = (select SUBSTRING_INDEX(date, '|', 1) from \
    hours where checkin_new.id = checkin.id);\
    update checkin_new set checkin_time = (select SUBSTRING_INDEX(date, '-', 1) from \
    hours where checkin_new.id = checkin.id);\
    update checkin_new set business_id = (select business_id from hours where checkin_new.id = checkin.id);\
    update checkin_new set count = (select count from checkin where checkin_new.id = checkin.id);"
executeQuery(yelp_conn, query_4)
query_5 =  "create index idx_day_of_week on hours(day_of_week)\
            create index idx_opening_time on hours(opening_time)\
            create index idx_closing_time on hours(closing_time)"
executeQuery(yelp_conn, query_5)